# INSTALADORES

In [50]:
!apt-get update
!pip install selenium webdriver-manager flask

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


# BIBLIOTECAS Y CONFIGURACIONES

In [51]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

import csv
from datetime import datetime

# Configuración más detallada de las opciones de Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')

# Configurar la ruta del driver
service = Service('/usr/bin/chromedriver')

In [52]:
from google.colab import drive
# Configuración específica para Colab
drive.mount('/content/drive')  # Montar Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# PAGE: RATINGS

In [62]:
# Inicializar el webdriver
driver = webdriver.Chrome(options=chrome_options)

# Listas para almacenar datos de todas las páginas
all_ratings = []
all_usernames = []
all_dates = []
all_comments = []

In [63]:
# Número total de páginas a recolectar
total_pages = 3  # Puedes ajustar este valor según necesites

# Recolectar datos de las páginas
for page in range(1, total_pages + 1):
    if page == 1:
        # Configuración inicial para la primera página
        driver.get(f'https://boardgamegeek.com/boardgame/295947/cascadia/ratings?pageid={page}')
        time.sleep(5)

        try:
            # Configurar paginación a 100 items
            boton_dropdown = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "per-page-dropdown"))
            )
            boton_dropdown.click()
            print("Botón dropdown clickeado correctamente")
            time.sleep(1)

            boton_opcion = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'ng-binding') and text()='100']"))
            )
            boton_opcion.click()
            print("Opción 100 seleccionada correctamente")
            time.sleep(3)

        except Exception as e:
            print(f"Error al interactuar con los botones: {str(e)}")
    else:
        # Navegación para páginas subsiguientes
        try:
            page_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f"//a[@class='ng-binding' and text()='{page}']"))
            )
            page_button.click()
            print(f"Navegando a página {page}")
            time.sleep(5)  # Esperar a que cargue la nueva página

        except Exception as e:
            print(f"Error al navegar a página {page}: {str(e)}")
            continue  # Continuar con la siguiente página si hay error

    # Extracción de datos (manteniendo tu lógica original)
    elements = driver.find_elements(By.CLASS_NAME, "summary-item-callout")
    ratings = [e.text.strip() for e in elements]
    all_ratings.extend(ratings)

    user_elements = driver.find_elements(By.CLASS_NAME, "comment-header-user")
    usernames = [user.text.strip() for user in user_elements]
    all_usernames.extend(usernames)

    date_elements = driver.find_elements(By.CSS_SELECTOR, ".comment-header-timestamp .ng-binding")
    dates = [date.text.strip() for date in date_elements if date.text.strip()]
    all_dates.extend(dates)

    comment_elements = driver.find_elements(By.CSS_SELECTOR, ".expandable-body p.mb-0.ng-binding.ng-scope")
    comments = [comment.text.strip() if comment.text.strip() else "NaN" for comment in comment_elements]
    all_comments.extend(comments)

    print(f"Datos de la página {page} recolectados")

# Cerrar el navegador
driver.quit()

Botón dropdown clickeado correctamente
Opción 100 seleccionada correctamente
Datos de la página 1 recolectados
Navegando a página 2
Datos de la página 2 recolectados
Navegando a página 3
Datos de la página 3 recolectados
Navegando a página 4
Datos de la página 4 recolectados
Navegando a página 5
Datos de la página 5 recolectados
Navegando a página 6
Datos de la página 6 recolectados
Navegando a página 7
Datos de la página 7 recolectados


In [68]:
# ----------------------------
# GUARDADO EN CSV
# ----------------------------
# Asegurar que todas las listas tienen igual longitud
max_length = max(len(all_ratings), len(all_usernames), len(all_dates), len(all_comments))
all_ratings += ['NaN'] * (max_length - len(all_ratings))  # Usando += en lugar de extend
all_usernames += ['NaN'] * (max_length - len(all_usernames))
all_dates += ['NaN'] * (max_length - len(all_dates))
all_comments += ['NaN'] * (max_length - len(all_comments))

# Combinar los datos
data = list(zip(all_ratings, all_usernames, all_dates, all_comments))

# Ruta en Google Drive (verifica que la carpeta exista)
file_path = '/content/drive/MyDrive/TUIA/NLP/NLP_CASCADIA/02_DATOS-EXTRAIDOS/cascadia_ratings.csv'

# Escribir el archivo CSV con manejo de errores
try:
    with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        # Sugerencia: usar nombres de columnas más descriptivos
        writer.writerow(['rating', 'username', 'date', 'comment'])
        writer.writerows(data)

    print(f"\nArchivo guardado exitosamente en:\n{file_path}")
    print(f"Registros guardados: {len(data)}")

except Exception as e:
    print(f"\nError al guardar el archivo: {str(e)}")
    print("Verifica:")
    print("1. Que Google Drive esté montado correctamente")
    print("2. Que la ruta de carpetas exista")
    print("3. Que tengas permisos de escritura")


Archivo guardado exitosamente en:
/content/drive/MyDrive/TUIA/NLP/NLP_CASCADIA/02_DATOS-EXTRAIDOS/cascadia_ratings.csv
Registros guardados: 350


# PAGE: VERSIONS

In [ ]:
# Inicializar el webdriver
driver = webdriver.Chrome(options=chrome_options)

# Navegar a la página de YouTube
driver.get('https://boardgamegeek.com/boardgame/295947/cascadia/versions')

# Esperar que la página cargue
time.sleep(5)

# ----------------------------
# Interacción con los botones de paginación
# ----------------------------

try:
    # Localizar y hacer clic en el botón 1 (dropdown)
    boton_dropdown = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "per-page-dropdown"))
    )
    boton_dropdown.click()
    print("Botón dropdown clickeado correctamente")

    # Esperar breve momento para que aparezca el menú desplegable
    time.sleep(1)

    # Localizar y hacer clic en el botón 2 (opción 25)
    # Usamos XPath para encontrar el botón con el texto exacto "25"
    boton_opcion = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'ng-binding') and text()='25']"))
    )
    boton_opcion.click()
    print("Opción 25 seleccionada correctamente")

    # Esperar a que se actualice la página
    time.sleep(3)

except Exception as e:
    print(f"Error al interactuar con los botones: {str(e)}")


Botón dropdown clickeado correctamente
Opción 25 seleccionada correctamente


In [ ]:
# Extraer todos los elementos con la clase "media-body"
media_bodies = driver.find_elements(By.CLASS_NAME, "media-body")

# Procesar cada elemento media-body
versions = []
for media_body in media_bodies:
    # Obtener todo el texto dentro del elemento y sus hijos
    full_text = media_body.text.strip()

    # Reemplazar saltos de línea por punto y coma para unificar
    processed_text = full_text.replace('\n', '; ')

    # Eliminar posibles puntos y coma duplicados
    processed_text = '; '.join([part.strip() for part in processed_text.split(';') if part.strip()])

    versions.append(processed_text)

# Cerrar el navegador
driver.quit()

# Imprimir los resultados
for i, result in enumerate(versions, 1):
    print(f"Media-body {i}:")
    print(result)
    print("\n" + "-"*80 + "\n")

Media-body 1:
Cascadia ‐ Baltic edition (2023); Publisher: Alderac Entertainment Group, Flatout Games, Terra Publica; Artist: Beth Sobel; Languages: Estonian, Latvian, Lithuanian; Status: Released; More options; Add To

--------------------------------------------------------------------------------

Media-body 2:
Cascadia ‐ Salta da Caixa Portuguese edition (2023); Publisher: Alderac Entertainment Group, Flatout Games, Salta da Caixa; Artist: N/A; Languages: Portuguese; More options; Add To

--------------------------------------------------------------------------------

Media-body 3:
Cascadia ‐ Turkish edition (2023); Publisher: Alderac Entertainment Group, Flatout Games, NeoTroy Games; Artist: N/A; Languages: Turkish; More options; Add To

--------------------------------------------------------------------------------

Media-body 4:
Каскадия ‐ Bulgarian edition (2022); Publisher: Alderac Entertainment Group, Fantasmagoria, Flatout Games; Artist: Beth Sobel; Languages: Bulgarian; S

In [ ]:
# ----------------------------
# GUARDADO EN TXT
# ----------------------------

# Guardar en Google Drive
with open('/content/drive/MyDrive/TUIA/NLP/NLP_CASCADIA/02_DATOS-EXTRAIDOS/cascadia_versions.txt', 'w', encoding='utf-8') as f:
    for i, version in enumerate(versions, 1):
        f.write(version + "\n")

print("Datos guardados exitosamente en 'cascadia_versions.txt'")

Datos guardados exitosamente en 'cascadia_versions.txt'
